In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.color import lab2rgb
from skimage.color import rgb2lab
import sys
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [2]:
OUTPUT_TEMPLATE = (
    'Bayesian classifier: {bayes_rgb:.3g} {bayes_lab:.3g}\n'
    'kNN classifier:      {knn_rgb:.3g} {knn_lab:.3g}\n'
    'SVM classifier:      {svm_rgb:.3g} {svm_lab:.3g}\n'
)

In [3]:
COLOUR_RGB = {
    'red': (255, 0, 0),
    'orange': (255, 114, 0),
    'yellow': (255, 255, 0),
    'green': (0, 230, 0),
    'blue': (0, 0, 255),
    'purple': (187, 0, 187),
    'brown': (117, 60, 0),
    'pink': (255, 187, 187),
    'black': (0, 0, 0),
    'grey': (150, 150, 150),
    'white': (255, 255, 255),
}
name_to_rgb = np.vectorize(COLOUR_RGB.get, otypes=[np.uint8, np.uint8, np.uint8])

In [4]:
data = pd.read_csv('colour-data.csv')
X = data[['R', 'G', 'B']].values / 255
y = data['Label'].values

In [51]:
# TODO: create some models
X_train, X_test, y_train, y_test = train_test_split(X, y)

bayes_rgb_model = GaussianNB()
bayes_lab_model = make_pipeline(
    FunctionTransformer(RGBtoLAB),
    GaussianNB()
)

knn_rgb_model = KNeighborsClassifier(n_neighbors=9)
knn_lab_model = make_pipeline(
    FunctionTransformer(RGBtoLAB),
    KNeighborsClassifier(n_neighbors=9)
)

svc_rgb_model = SVC(kernel='linear', C=0.1)
svc_lab_model = make_pipeline(
    FunctionTransformer(RGBtoLAB),
    SVC(kernel='linear', C=0.1)
)

In [52]:
# train each model and output image of predictions
models = [bayes_rgb_model, bayes_lab_model, knn_rgb_model, knn_lab_model, svc_rgb_model, svc_lab_model]
plt.close('all')
for i, m in enumerate(models):  # yes, you can leave this loop in if you want.
    m.fit(X_train, y_train)
    plot_predictions(m)
    plt.savefig('predictions-%i.png' % (i,))

In [50]:
print(OUTPUT_TEMPLATE.format(
    bayes_rgb=bayes_rgb_model.score(X_test, y_test),
    bayes_lab=bayes_lab_model.score(X_test, y_test),
    knn_rgb=knn_rgb_model.score(X_test, y_test),
    knn_lab=knn_lab_model.score(X_test, y_test),
    svm_rgb=svc_rgb_model.score(X_test, y_test),
    svm_lab=svc_lab_model.score(X_test, y_test),
 ))

Bayesian classifier: 0.537 0.597
kNN classifier:      0.725 0.736
SVM classifier:      0.557 0.717



In [6]:
def RGBtoLAB(X):
    X = X.reshape(1,-1,3)
    X = rgb2lab(X)
    X = X.reshape(-1,3)
    return X

In [9]:
def plot_predictions(model, lum=71, resolution=256):
    """
    Create a slice of LAB colour space with given luminance; predict with the model; plot the results.
    """
    wid = resolution
    hei = resolution
    n_ticks = 5

    # create a hei*wid grid of LAB colour values, with L=lum
    ag = np.linspace(-100, 100, wid)
    bg = np.linspace(-100, 100, hei)
    aa, bb = np.meshgrid(ag, bg)
    ll = lum * np.ones((hei, wid))
    lab_grid = np.stack([ll, aa, bb], axis=2)

    # convert to RGB for consistency with original input
    X_grid = lab2rgb(lab_grid)

    # predict and convert predictions to colours so we can see what's happening
    y_grid = model.predict(X_grid.reshape((wid*hei, 3)))
    pixels = np.stack(name_to_rgb(y_grid), axis=1) / 255
    pixels = pixels.reshape((hei, wid, 3))

    # plot input and predictions
    plt.figure(figsize=(10, 5))
    plt.suptitle('Predictions at L=%g' % (lum,))
    plt.subplot(1, 2, 1)
    plt.title('Inputs')
    plt.xticks(np.linspace(0, wid, n_ticks), np.linspace(-100, 100, n_ticks))
    plt.yticks(np.linspace(0, hei, n_ticks), np.linspace(-100, 100, n_ticks))
    plt.xlabel('A')
    plt.ylabel('B')
    plt.imshow(X_grid.reshape((hei, wid, 3)))

    plt.subplot(1, 2, 2)
    plt.title('Predicted Labels')
    plt.xticks(np.linspace(0, wid, n_ticks), np.linspace(-100, 100, n_ticks))
    plt.yticks(np.linspace(0, hei, n_ticks), np.linspace(-100, 100, n_ticks))
    plt.xlabel('A')
    plt.imshow(pixels)